# Amateur Hour - Market & News Neural Network
### Starter Kernel by ``Magichanics`` 
*([GitHub](https://github.com/Magichanics) - [Kaggle](https://www.kaggle.com/magichanics))*

With more features from public kernels, as well as the idea of using Neural Networks for modelling, I've decided to do some experimenting myself in hopes of producing the best results. Feel free to post suggestions or criticisms!

In [1]:
import numpy as np
import pandas as pd
import os
import gc

# import environment for data
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()

Loading the data... This could take a minute.
Done!


In [13]:
(market_train_df, news_train_df) = env.get_training_data()
market_train_df = market_train_df.tail(40_000)
news_train_df = news_train_df.tail(100_000)

In [20]:
test_cols = [ 'volume', 'close', 'open', 'returnsClosePrevRaw1',
 'returnsOpenPrevRaw1', 'returnsClosePrevMktres1', 'returnsOpenPrevMktres1',
'returnsClosePrevRaw10', 'returnsOpenPrevRaw10', 'returnsClosePrevMktres10',
 'returnsOpenPrevMktres10']

In [35]:
from sklearn.preprocessing import StandardScaler

# scale numerical columns
scaler = StandardScaler()

X_train = scaler.fit_transform(market_train_df[test_cols].fillna(0))

In [36]:
y_train = market_train_df['returnsOpenNextMktres10']

In [37]:
X_train.shape

(40000, 11)

In [38]:
def fixed_train_test_split(X, y, train_size):
    
    # round train size
    train_size = int(train_size * len(X))
    
    # split data
    X_train, y_train = X[train_size:], y[train_size:]
    X_valid, y_valid = X[:train_size], y[:train_size]
    
    return X_train, y_train, X_valid, y_valid

In [39]:
X_train, y_train, X_valid, y_valid = fixed_train_test_split(X_train, y_train, 5000)

In [30]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Concatenate, Flatten, BatchNormalization
from keras.losses import binary_crossentropy, mse

numerical_inputs = Input(shape=(11,), name='num')
numerical_logits = numerical_inputs
numerical_logits = BatchNormalization()(numerical_logits)

numerical_logits = Dense(128,activation='relu')(numerical_logits)
numerical_logits = Dense(64,activation='relu')(numerical_logits)

out = Dense(1, activation='sigmoid')(numerical_logits)

model = Model(inputs=numerical_inputs, outputs=out)
model.compile(optimizer='adam',loss=binary_crossentropy)

In [44]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

model.fit(X_train,y_train.astype(int),
          validation_data=(X_valid,y_valid.astype(int)),
          epochs=2,
          verbose=0) 

In [45]:
model.predict(X_valid)

array([[0.4688365 ],
       [0.49731448],
       [0.4594429 ],
       ...,
       [0.39670113],
       [0.37210885],
       [0.4968616 ]], dtype=float32)

**Sources:**
* [Market Data NN Baseline - Dieter](https://www.kaggle.com/christofhenkel/market-data-nn-baseline)